In [1]:
from sudoku import sudoku
import json

In [2]:
with open('sudoku_01.json') as f:
    d1 = json.load(f)
with open('sudoku_02.json') as f:
    d2 = json.load(f)

In [3]:
d1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [4]:
d2

[[8, 1, 0, 0, 3, 0, 0, 2, 7],
 [0, 6, 2, 0, 0, 0, 0, 9, 0],
 [0, 7, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 6, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 4],
 [0, 0, 8, 0, 0, 5, 0, 7, 0],
 [0, 0, 0, 0, 0, 0, 0, 8, 0],
 [0, 0, 0, 0, 1, 0, 7, 5, 0],
 [0, 0, 0, 0, 7, 0, 0, 4, 2]]

In [5]:
s1 = sudoku(3, d1)
s1.solve()

[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 0 2 8 0]
 [0 0 0 4 1 9 0 0 5]
 [0 0 0 0 8 0 0 7 9]] 44 5
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 7 2 8 0]
 [0 0 0 4 1 9 0 0 5]
 [0 0 0 0 8 0 0 7 9]] 65 7
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 7 2 8 4]
 [0 0 0 4 1 9 0 0 5]
 [0 0 0 0 8 0 0 7 9]] 68 4
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 0 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 7 2 8 4]
 [0 0 0 4 1 9 0 3 5]
 [0 0 0 0 8 0 0 7 9]] 77 3
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 0]
 [8 0 0 0 6 0 0 0 3]
 [4 2 0 8 5 3 0 0 1]
 [7 0 0 0 2 0 0 0 6]
 [0 6 0 0 0 7 2 8 4]
 [0 0 0 4 1 9 0 3 5]
 [0 0 0 0 8 0 0 7 9]] 41 2
[[5 3 0 0 7 0 0 0 0]
 [6 

Found solution


[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [6]:
s2 = sudoku(3, d2)
s2.solve()

[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 0 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 37 3
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 47 6
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 27 1
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 9]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 58 9
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 2 7 9]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 56 2
[[8 1 0 0 3 0 0 2 7]
 [0 

solvable
Fixing state 8 in position 24
[[8 1 9 4 3 6 5 2 7]
 [3 6 2 7 5 1 4 9 8]
 [4 7 5 2 8 0 3 1 6]
 [0 0 4 6 0 0 1 3 5]
 [0 0 7 3 0 0 8 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 3 5 0 0 9 8 1]
 [0 0 6 8 1 0 7 5 3]
 [5 8 1 9 7 3 6 4 2]] 24 8
[[8 1 9 4 3 6 5 2 7]
 [3 6 2 7 5 1 4 9 8]
 [4 7 5 2 8 9 3 1 6]
 [0 0 4 6 0 0 1 3 5]
 [0 0 7 3 0 0 8 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 3 5 0 0 9 8 1]
 [0 0 6 8 1 0 7 5 3]
 [5 8 1 9 7 3 6 4 2]] 25 9
[[8 1 9 4 3 6 5 2 7]
 [3 6 2 7 5 1 4 9 8]
 [4 7 5 2 8 9 3 1 6]
 [0 0 4 6 0 0 1 3 5]
 [0 0 7 3 0 2 8 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 3 5 0 0 9 8 1]
 [0 0 6 8 1 0 7 5 3]
 [5 8 1 9 7 3 6 4 2]] 45 2
[[8 1 9 4 3 6 5 2 7]
 [3 6 2 7 5 1 4 9 8]
 [4 7 5 2 8 9 3 1 6]
 [0 0 4 6 9 0 1 3 5]
 [0 0 7 3 0 2 8 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 3 5 0 0 9 8 1]
 [0 0 6 8 1 0 7 5 3]
 [5 8 1 9 7 3 6 4 2]] 34 9
[[8 1 9 4 3 6 5 2 7]
 [3 6 2 7 5 1 4 9 8]
 [4 7 5 2 8 9 3 1 6]
 [0 0 4 6 9 0 1 3 5]
 [0 0 7 3 9 2 8 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 3 5 0 0 9 8 1]
 [0 0 6 8 1 0 7 5 3]
 [5 8 1 9 7 3

[[8, 1, 9, 4, 3, 6, 5, 2, 7],
 [3, 6, 2, 7, 5, 1, 4, 9, 8],
 [4, 7, 5, 2, 9, 8, 3, 1, 6],
 [2, 9, 4, 6, 8, 7, 1, 3, 5],
 [1, 5, 7, 3, 2, 9, 8, 6, 4],
 [6, 3, 8, 1, 4, 5, 2, 7, 9],
 [7, 2, 3, 5, 6, 4, 9, 8, 1],
 [9, 4, 6, 8, 1, 2, 7, 5, 3],
 [5, 8, 1, 9, 7, 3, 6, 4, 2]]